In [1]:
import os
import pandas as pd 
import numpy as np 
import scipy as scp
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

import statsmodels.api as sm
import matplotlib.pyplot as plt

#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# 📂DATA : 2 Classes

In [33]:
fold = 3

In [34]:
path = "/home/kannika/code/Rheology2023/Rheology_Blood/DataBlood_Viscosity_TrainML_3Fold_EMclass.csv"
data_feature = pd.read_csv(path)
print(data_feature.shape)
print("-"*100)
print(f"All Fold : {set(data_feature.fold)}")
## Split Train data Set
feature_train = data_feature[data_feature["fold"]!=fold].reset_index(drop=True)
print(f"Train Set : Fold ==> {set(feature_train.fold)}")
print("Train = ", feature_train.shape)
## Split Valid data Set
feature_test = data_feature[data_feature["fold"]==fold].reset_index(drop=True)
print(f"Valiadtion Set : Fold ==> {set(feature_test.fold)}")
print("Validation = ", feature_test.shape)
## Print DataFrame
feature_train.head()

(33, 11)
----------------------------------------------------------------------------------------------------
All Fold : {1, 2, 3}
Train Set : Fold ==> {1, 2}
Train =  (22, 11)
Valiadtion Set : Fold ==> {3}
Validation =  (11, 11)


,Unnamed: 0,Unnamed: 0.1,Code,classes,subclass,classes_binary,MCV,MCH,Hb,typeBEvsBM,fold
0,0,42,HN44,HN,Splenectomy,1.0,61.7,19.7,7.0,E,1
1,1,17,HN21,HN,No_Splenectomy,1.0,64.8,19.7,6.5,E,1
2,2,8,HN13,HN,Splenectomy,1.0,72.6,23.4,5.8,E,1
3,3,43,HN29,HN,No_Splenectomy,1.0,64.2,19.1,6.6,E,1
4,4,50,HN39,HN,Splenectomy,1.0,76.4,22.5,6.4,E,1


In [35]:
X_train = feature_train[['MCV','MCH','Hb']]
y_train = feature_train["typeBEvsBM"]
print(X_train.shape)
print(y_train.shape)
print(y_train[0])

(22, 3)
(22,)
E


In [36]:
print(len(list(set(y_train))))
print(set(y_train))

2
{'M', 'E'}


## 🩸 Parameter: Decision Tree

In [37]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings('ignore')

In [38]:
Dtree = DecisionTreeClassifier(random_state=1)

params = {"splitter":["best","random"],
          'max_depth': np.linspace(1, 7, 7, endpoint=True),
          'min_samples_leaf':  np.linspace(0.1, 0.5, 5, endpoint=True),
          'min_samples_split': np.linspace(0.1, 1.0, 10, endpoint=True),
          'criterion': ["gini", "entropy"]}

tuning_Dtree = GridSearchCV(estimator=Dtree, param_grid=params, cv=10, n_jobs=-1, verbose=1, scoring = "accuracy")

# fitting the model for grid search 
best_Dtree = tuning_Dtree.fit(X_train, y_train)

Fitting 10 folds for each of 1400 candidates, totalling 14000 fits


In [39]:
# view the results as a pandas DataFrame
best_Dtreedf = pd.DataFrame(best_Dtree.cv_results_)[['rank_test_score', 'mean_test_score', 'std_test_score', 'params']]
best_Dtreedf = best_Dtreedf.sort_values(by='rank_test_score', ascending=True)
best_Dtreedf = best_Dtreedf.reset_index(drop=True)
best_Dtreedf.head()

,rank_test_score,mean_test_score,std_test_score,params
0,1,0.783333,0.224227,"{'criterion': 'entropy', 'max_depth': 2.0, 'mi..."
1,1,0.783333,0.224227,"{'criterion': 'entropy', 'max_depth': 4.0, 'mi..."
2,1,0.783333,0.224227,"{'criterion': 'entropy', 'max_depth': 2.0, 'mi..."
3,1,0.783333,0.224227,"{'criterion': 'gini', 'max_depth': 5.0, 'min_s..."
4,1,0.783333,0.224227,"{'criterion': 'entropy', 'max_depth': 2.0, 'mi..."


In [40]:
# examine the first result
print("**examine the first result","\n")

print(best_Dtree.cv_results_['params'][0])
print(best_Dtree.cv_results_['mean_test_score'][0])

# print the array of mean scores only
print("\n","**print the array of mean scores only","\n")

grid_mean_scores = best_Dtree.cv_results_['mean_test_score']
print(grid_mean_scores)

# examine the best model
print("\n","**examine the best model","\n")

print(best_Dtree.best_score_)
print(best_Dtree.best_params_)
print(best_Dtree.best_estimator_)

**examine the first result 

{'criterion': 'gini', 'max_depth': 1.0, 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'splitter': 'best'}
0.6166666666666666

 **print the array of mean scores only 

[0.61666667 0.78333333 0.61666667 ... 0.73333333 0.53333333 0.73333333]

 **examine the best model 

0.7833333333333333
{'criterion': 'gini', 'max_depth': 1.0, 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'splitter': 'random'}
DecisionTreeClassifier(max_depth=1.0, min_samples_leaf=0.1,
                       min_samples_split=0.1, random_state=1,
                       splitter='random')


In [41]:
#Print the tured parameters and score
print("Tuned Decision Tree Parameters: {}".format(best_Dtree.best_params_))
print("Best score is {}".format(best_Dtree.best_score_))

Tuned Decision Tree Parameters: {'criterion': 'gini', 'max_depth': 1.0, 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'splitter': 'random'}
Best score is 0.7833333333333333


In [43]:
best_params_ =  best_Dtree.best_params_
criterion_ = best_params_['criterion'] 
max_depth_ = best_params_['max_depth'] 
min_samples_leaf_ = best_params_['min_samples_leaf']   
min_samples_split_ = best_params_['min_samples_split']
splitter_ = best_params_['splitter']

> ## 🚀 Fit Model and setting parameters

In [44]:
## **- train Fit Model XGBOOST
DtreeClassifier = DecisionTreeClassifier(random_state=1, criterion=criterion_, max_depth=max_depth_, min_samples_leaf=min_samples_leaf_,
                                 min_samples_split=min_samples_split_, splitter=splitter_)
DtreeClassifierModel = DtreeClassifier.fit(X_train, y_train) # train XGBOOST model
DtreeClassifierModel

DecisionTreeClassifier(max_depth=1.0, min_samples_leaf=0.1,
                       min_samples_split=0.1, random_state=1,
                       splitter='random')

In [45]:
### Testing Check Parameter
print(DtreeClassifierModel.get_params())

{'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 1.0, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'min_weight_fraction_leaf': 0.0, 'random_state': 1, 'splitter': 'random'}


In [30]:
# from sklearn import tree
# tree.plot_tree(DtreeClassifierModel);

> ## #️⃣ Save model

In [46]:
nameclass = "typeBEvsBM"

In [47]:
#Save Model
import os
import pickle
import imageio

save_pathimg = f'/media/tohn/HDD/rheology2023/ML_Model/Blood_Viscosity/{nameclass}_Classes/fold{fold}'
##**Mkdir Directory 
os.makedirs(save_pathimg, exist_ok=True)     
filename = f"DtreeClassifier_Model_{nameclass}_Class_fold{fold}.pkl"
Model2Save =  f"{save_pathimg}/{filename}"
print(f"[INFO]: Done!! Save Model as : {Model2Save}")

with open(Model2Save, 'wb') as file:
    pickle.dump(DtreeClassifierModel, file)

[INFO]: Done!! Save Model as : /media/tohn/HDD/rheology2023/ML_Model/Blood_Viscosity/typeBEvsBM_Classes/fold3/DtreeClassifier_Model_typeBEvsBM_Class_fold3.pkl
